In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [2]:
df = yf.download('EURUSD=X', interval='15m', period='60d', multi_level_index=False)

C:\Users\Ido Sabach\AppData\Local\Temp\ipykernel_16080\3647498583.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('EURUSD=X', interval='15m', period='60d', multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


In [3]:
df

,Close,High,Low,Open,Volume
Datetime,,,,,
2025-08-24 23:00:00+00:00,1.171509,1.171921,1.171509,1.171921,0
2025-08-24 23:15:00+00:00,1.171097,1.171646,1.171097,1.171509,0
2025-08-24 23:30:00+00:00,1.171234,1.171234,1.171097,1.171234,0
2025-08-24 23:45:00+00:00,1.171234,1.171372,1.171097,1.171234,0
2025-08-25 00:00:00+00:00,1.170549,1.170960,1.170412,1.170960,0
...,...,...,...,...,...
2025-11-14 21:15:00+00:00,1.162520,1.162656,1.162385,1.162385,0
2025-11-14 21:30:00+00:00,1.162385,1.162520,1.162115,1.162520,0
2025-11-14 21:45:00+00:00,1.162520,1.162656,1.162250,1.162385,0


In [6]:
df.to_csv('data/df60d.csv')

In [61]:
df = pd.read_csv('data/df60d.csv', index_col=0, parse_dates=True)

class DataLoad():
  def __init__(self, df):
    self.raw_data = df
    self.data = df

  def process_data(self):
    df = self.data.copy()
    df = df.rename(columns={'Close':'price'})
    df['returns'] = np.log(df['price'] / df['price'].shift(1))
    df.index = pd.to_datetime(df.index)
    df['hour'] = df.index.hour
    df['date'] = df.index.date   
    df.dropna(inplace=True)
    self.data = df   
    return self.data

  def split_sessions(self):
    df = self.data.copy()
    self.df_asia = df[(df['hour'] >= 2) & (df['hour'] < 10)]
    self.df_london = df[(df['hour'] >= 10) & (df['hour'] < 18)]
    self.df_ny = df[(df['hour'] >= 15) & (df['hour'] <= 23)]
    self.df_deadzone = df[(df['hour'] >= 0) & (df['hour'] < 2)]

    sessions = {
      'asia' : self.df_asia,
      'london' : self.df_london,
      'ny' : self.df_ny,
      'deadzone' : self.df_deadzone
    }

    return sessions
  
  def run(self):
    self.process_data()
    self.split_sessions()
  

In [62]:
eur = DataLoad(df)
eur.data

,Close,High,Low,Open,Volume
Datetime,,,,,
2025-08-24 23:00:00+00:00,1.171509,1.171921,1.171509,1.171921,0
2025-08-24 23:15:00+00:00,1.171097,1.171646,1.171097,1.171509,0
2025-08-24 23:30:00+00:00,1.171234,1.171234,1.171097,1.171234,0
2025-08-24 23:45:00+00:00,1.171234,1.171372,1.171097,1.171234,0
2025-08-25 00:00:00+00:00,1.170549,1.170960,1.170412,1.170960,0
...,...,...,...,...,...
2025-11-14 21:15:00+00:00,1.162520,1.162656,1.162385,1.162385,0
2025-11-14 21:30:00+00:00,1.162385,1.162520,1.162115,1.162520,0
2025-11-14 21:45:00+00:00,1.162520,1.162656,1.162250,1.162385,0


In [63]:
eur.run()

In [64]:
eur.df_ny

,price,High,Low,Open,Volume,returns,hour,date
Datetime,,,,,,,,
2025-08-24 23:15:00+00:00,1.171097,1.171646,1.171097,1.171509,0,-0.000351,23,2025-08-24
2025-08-24 23:30:00+00:00,1.171234,1.171234,1.171097,1.171234,0,0.000117,23,2025-08-24
2025-08-24 23:45:00+00:00,1.171234,1.171372,1.171097,1.171234,0,0.000000,23,2025-08-24
2025-08-25 15:00:00+00:00,1.168907,1.169454,1.168770,1.169317,0,-0.000351,15,2025-08-25
2025-08-25 15:15:00+00:00,1.168634,1.168907,1.168497,1.168770,0,-0.000234,15,2025-08-25
...,...,...,...,...,...,...,...,...
2025-11-14 21:15:00+00:00,1.162520,1.162656,1.162385,1.162385,0,0.000116,21,2025-11-14
2025-11-14 21:30:00+00:00,1.162385,1.162520,1.162115,1.162520,0,-0.000116,21,2025-11-14
2025-11-14 21:45:00+00:00,1.162520,1.162656,1.162250,1.162385,0,0.000116,21,2025-11-14
